# AeroGuard Predictive Maintenance - End-to-End Demonstration

This notebook demonstrates the complete AeroGuard system:
1. Data processing with causal adjustments
2. Model training with probabilistic outputs
3. SHAP-based explainability
4. Maintenance alert generation
5. Work order creation
6. Feedback loop simulation

**Note**: This demo uses synthetic/mock data. For real deployment, use NASA C-MAPSS dataset.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from pathlib import Path

# AeroGuard modules
from data_processor import TurbofanDataProcessor
from causal_engine import CausalEngine
from aeroguard_model import AeroGuardModel, train_model
from explainability import ExplainabilityEngine, generate_alerts_for_dataset
from maintenance_integration import MaintenanceIntegration
from feedback_system import FeedbackSystem, simulate_maintenance_workflow
import config
from utils import setup_logging, plot_rul_prediction

# Setup
logger = setup_logging('INFO')
np.random.seed(42)
torch.manual_seed(42)

print("✓ AeroGuard modules loaded")

## Part 1: Data Processing with Causal Inference

In [ ]:
# Create synthetic turbofan data for demonstration
# In production, load real NASA C-MAPSS data:
# processor = TurbofanDataProcessor()
# data = processor.process_pipeline('data/train_FD001.txt')

def create_synthetic_data(num_units=50, max_cycles=200):
    """Generate synthetic turbofan data"""
    data_list = []
    
    for unit in range(1, num_units + 1):
        cycles = np.random.randint(100, max_cycles)
        
        for cycle in range(1, cycles + 1):
            # Operational settings
            op1 = np.random.uniform(0, 1)  # Altitude
            op2 = np.random.uniform(0, 1)  # Mach
            op3 = np.random.uniform(0, 1)  # Throttle
            
            # Degradation factor (increases with cycles)
            degradation = (cycles - cycle) / cycles  # 0 at failure, 1 at start
            
            # Sensors with degradation + environmental effects
            T2 = 500 + 50 * op1 + (1 - degradation) * 20 + np.random.normal(0, 2)
            T24 = 600 + 30 * op2 + (1 - degradation) * 25 + np.random.normal(0, 3)
            T30 = 1500 + 100 * op3 + (1 - degradation) * 50 + np.random.normal(0, 5)
            T50 = 1350 - 50 * op1 + (1 - degradation) * 60 + np.random.normal(0, 5)
            
            P2 = 14 - 5 * op1 + np.random.normal(0, 0.3)
            P15 = 21 + 3 * op2 + np.random.normal(0, 0.4)
            P30 = 550 + 50 * op3 + (1 - degradation) * 20 + np.random.normal(0, 2)
            
            Nf = 2400 + 200 * op3 + (1 - degradation) * 50 + np.random.normal(0, 10)
            Nc = 9000 + 500 * op3 + (1 - degradation) * 100 + np.random.normal(0, 20)
            
            data_list.append([
                unit, cycle, op1, op2, op3,
                T2, T24, T30, T50, P2, P15, P30,
                Nf, Nc,
                P30/P2,  # epr
                P30 - 10,  # Ps30
                0.01 * op3,  # phi
                Nf * 0.9, Nc * 0.9,  # NRf, NRc
                2.5, 0.05, 100,  # BPR, farB, htBleed
                Nf, Nf * 0.9,  # Nf_dmd, PCNfR_dmd
                1.5, 1.2  # W31, W32
            ])
    
    df = pd.DataFrame(data_list, columns=config.SENSOR_COLUMNS)
    return df

print("Creating synthetic turbofan data...")
synthetic_df = create_synthetic_data(num_units=30, max_cycles=150)
print(f"✓ Generated {len(synthetic_df)} observations for {synthetic_df['unit_number'].nunique()} engines")
synthetic_df.head()

In [ ]:
# Initialize processors
processor = TurbofanDataProcessor()
causal_engine = CausalEngine()

# Calculate RUL
df_with_rul = processor.calculate_rul(synthetic_df)

# Apply causal adjustments
df_causal = causal_engine.process_data_with_causal_adjustments(df_with_rul)

# Engineer features
df_features = processor.engineer_features(df_causal)

print(f"✓ Features engineered: {len(df_features.columns)} total columns")
print(f"  Original sensors: {len(config.SENSOR_COLUMNS)}")
print(f"  Engineered features: {len(df_features.columns) - len(config.SENSOR_COLUMNS)}")

In [ ]:
# Build sequences for LSTM
X_sequences, y_rul, feature_names = processor.build_sequences(df_features)

# Normalize
X_normalized = processor.normalize_features(X_sequences, fit=True)

# Train/val split
X_train, X_val, y_train, y_val = processor.prepare_train_test_split(X_normalized, y_rul)

print(f"✓ Data prepared for training:")
print(f"  Train: {X_train.shape}")
print(f"  Val: {X_val.shape}")
print(f"  Features: {len(feature_names)}")

## Part 2: Model Training

In [ ]:
# Initialize AeroGuard model
model = AeroGuardModel(
    num_features=len(feature_names),
    causal_hidden_dim=64,
    lstm_hidden_dim=128,
    lstm_layers=2,
    attention_heads=4
)

print(f"✓ Model initialized")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Train model (reduced epochs for demo)
history = train_model(
    model,
    train_data={'X': X_train, 'y': y_train},
    val_data={'X': X_val, 'y': y_val},
    num_epochs=20,  # Increase to 100+ for production
    batch_size=32,
    learning_rate=0.001
)

print(f"\n✓ Training complete!")
print(f"  Best RMSE: {history['best_rmse']:.2f} cycles")
print(f"  Best epoch: {history['best_epoch'] + 1}")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train Loss')
ax1.plot(history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training History - Loss')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(history['val_rmse'], label='Val RMSE', color='orange')
ax2.axhline(history['best_rmse'], color='red', linestyle='--', label=f'Best: {history['best_rmse']:.2f}')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('RMSE (cycles)')
ax2.set_title('Validation RMSE')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: Predictions with Uncertainty

In [ ]:
# Make predictions on validation set
model.eval()
X_val_tensor = torch.FloatTensor(X_val[:50])  # First 50 samples

# Get predictions with uncertainty (MC Dropout)
pred_mean, pred_std = model.predict_with_uncertainty(X_val_tensor, num_samples=50)

# Plot predictions
fig = plot_rul_prediction(
    actual_rul=y_val[:50],
    predicted_rul=pred_mean,
    uncertainty=pred_std
)
plt.show()

print(f"\n✓ Prediction statistics:")
print(f"  Mean RUL: {pred_mean.mean():.1f} ± {pred_std.mean():.1f} cycles")
print(f"  Actual RUL: {y_val[:50].mean():.1f} cycles")
print(f"  MAE: {np.abs(pred_mean - y_val[:50]).mean():.2f} cycles")

## Part 4: Explainability (SHAP)

In [ ]:
# Initialize explainability engine
explainer = ExplainabilityEngine(
    model=model,
    background_data=X_train[:100],  # Sample for SHAP
    feature_names=feature_names
)

# Pick a sample with low RUL for demonstration
low_rul_idx = np.argmin(y_val[:50])
sample_sequence = X_val[low_rul_idx]
sample_rul = y_val[low_rul_idx]

print(f"Analyzing Engine with RUL = {sample_rul:.0f} cycles...")

# Generate explanation
explanation = explainer.explain_prediction(
    sequence=sample_sequence,
    actual_rul=sample_rul
)

print(f"\n✓ Explanation generated:")
print(f"  Predicted RUL: {explanation['predicted_rul_mean']:.1f} ± {explanation['predicted_rul_std']:.1f}")
print(f"  Actual RUL: {explanation['actual_rul']:.1f}")
print(f"\n  Top Contributing Sensors:")
for feat in explanation['top_features']:
    print(f"    • {feat['name']}: {feat['contribution_pct']:.1f}%")

In [ ]:
# Visualize feature importance
fig = explainer.visualize_explanation(explanation)
plt.show()

## Part 5: Maintenance Alert Generation

In [ ]:
# Generate maintenance alert
alert = explainer.generate_maintenance_alert(
    engine_id=5,
    explanation=explanation,
    confidence_level='conservative'
)

# Display formatted alert
print(alert['alert_message'])

## Part 6: Work Order Generation

In [ ]:
# Initialize maintenance integration
maintenance = MaintenanceIntegration()

# Generate work order
work_order = maintenance.generate_work_order(
    alert=alert,
    aircraft_id='N54321',
    flights_scheduled=8,
    mechanic_id='TECH-ALPHA'
)

# Display formatted work order
print(maintenance.format_work_order_for_print(work_order))

print(f"\n✓ Work Order Details:")
print(f"  Priority: {work_order['priority']} (Score: {work_order['priority_score']:.2f})")
print(f"  Safe Flight Budget: {work_order['safe_flight_budget']:.0f} flights")
print(f"  Estimated Downtime: {work_order['estimated_downtime_hours']} hours")
print(f"  Parts Cost: ${work_order['total_parts_cost']:,.2f}")

## Part 7: Feedback Loop Simulation

In [ ]:
# Initialize feedback system
feedback_system = FeedbackSystem()

# Simulate maintenance workflow
workflow = simulate_maintenance_workflow(
    alert=alert,
    work_order=work_order,
    feedback_system=feedback_system,
    technician_id='TECH-BRAVO'
)

print(f"\n✓ Workflow Summary:")
print(f"  Alert ID: {workflow['alert_id']}")
print(f"  Outcome: {workflow['outcome']}")
print(f"  Technician: {workflow['technician_id']}")

In [ ]:
# Simulate multiple feedback submissions
print("Simulating 20 alert feedback submissions...\n")

for i in range(20):
    # Create mock alert
    mock_alert = {
        'alert_id': f'AG-DEMO-{i:03d}',
        'timestamp': pd.Timestamp.now().isoformat(),
        'engine_id': np.random.randint(1, 6),
        'rul_mean': np.random.uniform(30, 100),
        'rul_std': np.random.uniform(5, 15),
        'top_sensors': [('T50', 0.4), ('Nc', 0.3)],
        'recommended_action': 'Inspect turbine',
        'priority': np.random.choice(['CRITICAL', 'HIGH', 'MEDIUM'])
    }
    
    feedback_system.record_alert(mock_alert, f'WO-DEMO-{i:03d}')
    
    # Random outcome (80% confirmed, 15% false positive, 5% already known)
    outcome = np.random.choice(
        ['CONFIRMED', 'REJECTED_FALSE_POSITIVE', 'REJECTED_ALREADY_KNOWN'],
        p=[0.80, 0.15, 0.05]
    )
    
    feedback_system.submit_feedback(
        alert_id=mock_alert['alert_id'],
        outcome=outcome,
        technician_id=f'TECH-{np.random.randint(1, 5):03d}',
        technician_notes=f'Demo feedback for alert {i}'
    )

print("✓ Feedback simulation complete")

In [ ]:
# Get performance summary
performance = feedback_system.get_performance_summary()

print("\n" + "="*70)
print("AEROGUARD PERFORMANCE SUMMARY")
print("="*70)
print(f"Total Alerts Issued: {performance['total_alerts']}")
print(f"\nOutcome Breakdown:")
print(f"  ✓ Confirmed: {performance['confirmed']} ({performance['confirmed']/performance['total_alerts']*100:.1f}%)")
print(f"  ✗ False Positives: {performance['rejected_false_positive']} ({performance['false_positive_rate']*100:.1f}%)")
print(f"  ℹ Already Known: {performance['rejected_already_known']}")
print(f"  ⏸ Deferred: {performance['deferred']}")
print(f"\nPerformance Metrics:")
print(f"  Precision: {performance['precision']:.1%}")
print(f"  Target Precision: {performance['target_precision']:.0%}")
print(f"  Meets Target: {'✓ YES' if performance['meets_target_precision'] else '✗ NO'}")
print("="*70)

In [ ]:
# Check retraining readiness
is_ready, analytics = feedback_system.prepare_retraining_data(min_samples=15)

print(f"\n✓ Retraining Analysis:")
print(f"  Feedback Samples: {analytics['total_feedback_samples']}")
print(f"  Minimum Required: {analytics['min_required']}")
print(f"  Ready for Retraining: {'✓ YES' if is_ready else '✗ NO'}")

if is_ready:
    print(f"\n  → System ready for quarterly retraining with expert review")

In [ ]:
# Export feedback for expert review
export_path = Path('feedback_export.csv')
feedback_system.export_feedback_for_review(export_path)

# Display sample
feedback_df = pd.read_csv(export_path)
print(f"\n✓ Feedback exported to {export_path}")
print(f"\nSample records:")
feedback_df.head(10)

## Summary

This demonstration showed the complete AeroGuard workflow:

1. **✓ Data Processing**: Causal adjustments remove environmental noise
2. **✓ ML Model**: Bi-LSTM with causal attention predicts RUL with uncertainty
3. **✓ Explainability**: SHAP shows which sensors drove each prediction
4. **✓ Maintenance Integration**: Alerts converted to actionable work orders
5. **✓ Feedback Loop**: Non-punitive learning from technician feedback

### Key Metrics
- **Model RMSE**: ~{rmse} cycles on validation set
- **Alert Precision**: {precision}% (target: 75%)
- **System Status**: {status}

### Next Steps for Production
1. Train on full NASA C-MAPSS dataset (100+ epochs)
2. Integrate with real-time aircraft data streams
3. Deploy shadow mode alongside existing maintenance processes
4. Collect 3-6 months of feedback for calibration
5. Seek FAA/EASA advisory approval for supplemental use

---

**Remember**: AeroGuard is an *advisory system*. All maintenance decisions require certified technician approval per regulatory requirements.